In [1]:
import sys, os
PATH = os.path.dirname(os.path.abspath(os.curdir))
if PATH not in sys.path:
    sys.path.insert(0, PATH)

import logging

logging.basicConfig(level=logging.ERROR)
logging.getLogger("torch").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)

from pathos.multiprocessing import ProcessPool
import src.Simulator as sim_system
import src.Optimizer as opt
import src.SimGrad as sim_diff
import scipy as sp
import numpy as np
import torch
import tqdm


###* Create Simulator object
reactions_file = "../reactions/reactionsSimpleV1.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                    'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                    'COdb_S': 0.0}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O', 0.016),
    'fluxO2' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O2', 0.032),
    'fluxO3' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O3', 0.048),
    'fluxC':    lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'C', 0.012),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

output_folder_path = "../Buffer_Data"
exp_data_file = "Experimental_data_CO_Jorge.hdf5"
exp_file = os.path.join(output_folder_path, exp_data_file)

sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 + r_32*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_S]/dt = -CO_F*O_S*r_39 - O_F*O_S*r_7 + O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - O_S*r_16 - O_S*r_17 - O_S*r_37 - O_S*r_6 + r_3*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[Odb_S]/dt = -O_F*Odb_S*r_27 + O_F*Vdb_S*r_26 - Odb_S*r_23 - Odb_S*r_24 - Odb

In [2]:
###* Create optimization and diff objects with the proper bounds


lower_bounds = np.array([0.0, 0.0, 1e-8, 1e-8])
upper_bounds = np.array([25.0, 25.0, 1.0, 1.0])

##! define parameters to optimize
def func_optimization(params, flag='numpy'):
    
    E_d, E_d2, SF_1, SF_2 = params
    
    # SF_1, SF_2 = params
    
    E_d = lower_bounds[0] + (upper_bounds[0] - lower_bounds[0]) * E_d
    E_d = lower_bounds[1] + (upper_bounds[1] - lower_bounds[1]) * E_d
    
    SF_1 = lower_bounds[2] + (upper_bounds[2] - lower_bounds[2]) * SF_1
    SF_2 = lower_bounds[3] + (upper_bounds[3] - lower_bounds[3]) * SF_2
    
    A_d = 0.01634
    B_d = 1.67e-4
    if flag=='numpy':
        nu_d_mod = lambda T: 1e15 * (A_d + B_d * np.exp(E_d/(const_dict['R'] * T)))
        nu_d_mod2 = lambda T: 1e15 * (A_d + B_d * np.exp(E_d2/(const_dict['R'] * T)))
    elif flag=='torch':
        nu_d_mod = lambda T: 1e15 * (A_d + B_d * torch.exp(E_d/(const_dict['R'] * T)))
        nu_d_mod2 = lambda T: 1e15 * (A_d + B_d * torch.exp(E_d2/(const_dict['R'] * T)))
    else:
        raise ValueError(f"{flag} does not exist")
    

    
    # A_d = 0.01634
    # B_d = 1.67e-4
    # if flag=='numpy':
    #     nu_d_mod = lambda T: 1e15 * (A_d + B_d * np.exp(E_d/(const_dict['R'] * T)))
    #     nu_d_mod2 = lambda T: 1e15 * (A_d + B_d * np.exp(E_d2/(const_dict['R'] * T)))
    # elif flag=='torch':
    #     nu_d_mod = lambda T: 1e15 * (A_d + B_d * torch.exp(E_d/(const_dict['R'] * T)))
    #     nu_d_mod2 = lambda T: 1e15 * (A_d + B_d * torch.exp(E_d2/(const_dict['R'] * T)))
    # else:
    #     raise ValueError(f"{flag} does not exist")
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    # {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod2}},
    {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod2, "SF":SF_2}},
    {"id": 30, "rate": None, "model_dict": {"SF": SF_1}},
    ]
    
    return dict_mod_vec

##! define the default parameters
params_default = []
params_default_aux = list((19.75, 19.75, 1.0, 1.0))
for idx, param in enumerate(params_default_aux):
    value = (param - lower_bounds[idx])/(upper_bounds[idx] - lower_bounds[idx])
    params_default.append(value)

params_default = tuple(params_default)

# params_default = [19.75, 19.75, 1.0, 1.0]
# params_default = [1.0, 1.0]
# params_default = [19.75]

print("params_default: ", params_default)

def loss_function(exp, teo, flag='numpy'):
    
    func = ((teo-exp)**2)/(exp**2)
    if flag == 'numpy':
        return np.mean(func)
    elif flag == 'torch':
        return torch.mean(func)
    else:
        raise ValueError(f"{flag} does not exist")


optimizer = opt.Optimizer(sim, 
                        lambda params: func_optimization(params, 'numpy'), 
                        lambda exp, teo: loss_function(exp, teo, 'numpy')
                        )

diff = sim_diff.SimDiff(sim, 
                        lambda params: func_optimization(params, 'torch'),
                        params_default=params_default,
                        gamma_exp_data=sim.gamma_exp_data_arr,
                        loss_function=lambda exp, teo: loss_function(exp, teo, 'torch')
                        )


params_default:  (0.79, 0.79, 1.0, 1.0)


In [3]:


def loss_and_grads(params, opt_object, diff_object):
    loss_val, frac_solutions_arr, rates_arr, _, gammas_predicted_arr = opt_object.objective_function_diff(params)
    grad_val = diff_object.objective_function_grad(params, frac_solutions_arr, rates_arr, gammas_predicted_arr)
    # print("loss_val: ", loss_val, "grad_val: ", grad_val, "params: ", params)
    return loss_val, grad_val.detach().numpy()



params1 = np.array(params_default)
print(params1)
# params1[0] = 0.90
res1 = loss_and_grads(params1, optimizer, diff)

params2 = params1.copy()
params2[0] += 1e-4
print(params2)
res2 = loss_and_grads(params2, optimizer, diff)

params3 = params1.copy()
params3[0] -= 1e-4
print(params3)
res3 = loss_and_grads(params3, optimizer, diff)

print(res1)
print(res2)
print(res3)

# param_vec = np.linspace(0.79, 0.82, 10)
# for param in param_vec:
#     params = np.array(params_default)
#     params[0] = param
    
#     print(loss_and_grads(params, optimizer, diff), param)


#### some conditions are simply ill conditioned


[0.79 0.79 1.   1.  ]
Jx shape: torch.Size([8, 8]) Jθ shape: torch.Size([8, 4])
‖Jx‖₂ = 3.196584918917491e+88 cond(Jx)= -inf
resid:  1.3575214395500222e+75
Jx shape: torch.Size([8, 8]) Jθ shape: torch.Size([8, 4])
‖Jx‖₂ = 3.196584918917491e+88 cond(Jx)= -inf
resid:  1.3575214395500222e+75
Jx shape: torch.Size([8, 8]) Jθ shape: torch.Size([8, 4])
‖Jx‖₂ = 3.196584918917491e+88 cond(Jx)= -inf
resid:  1.3575214395500222e+75
Jx shape: torch.Size([8, 8]) Jθ shape: torch.Size([8, 4])
‖Jx‖₂ = 3.196584918917491e+88 cond(Jx)= -inf
resid:  1.3575214395500222e+75
Jx shape: torch.Size([8, 8]) Jθ shape: torch.Size([8, 4])
‖Jx‖₂ = 3.196584918917491e+88 cond(Jx)= -inf
resid:  1.3575214395500222e+75
Jx shape: torch.Size([8, 8]) Jθ shape: torch.Size([8, 4])
‖Jx‖₂ = 3.196584918917491e+88 cond(Jx)= -inf
resid:  1.3575214395500222e+75
Jx shape: torch.Size([8, 8]) Jθ shape: torch.Size([8, 4])
‖Jx‖₂ = 3.196584918917491e+88 cond(Jx)= inf
resid:  1.3575214395500222e+75
Jx shape: torch.Size([8, 8]) Jθ shape: to

In [4]:
print((res2[0]-res3[0])/(2e-4))

-1.0103029524088925e-10


In [5]:
ping()

NameError: name 'ping' is not defined

In [ ]:

def grads_wise(params, opt_object, diff_object):
    loss_val, frac_solutions_arr, rates_arr, _, gammas_predicted_arr = opt_object.objective_function_diff(params)
    grad_val = diff_object.objective_function_grad_element_wise(params, frac_solutions_arr, rates_arr, gammas_predicted_arr)
    return grad_val.numpy()


def create_subspaces(params, percent_info, opt_object, diff_object):

    grad_errors =  - grads_wise(params_default, opt_object, diff_object) / sim.gamma_exp_data_arr.reshape(-1, 1)
    F_matrix = np.matmul(grad_errors.T, grad_errors)

    eigenvalues, eigenvector = np.linalg.eigh(F_matrix)
    
    np.linalg.norm(eigenvector, axis=0)

    idx = np.argsort(np.abs(eigenvalues))[::-1]
    eigvals_sorted = eigenvalues[idx]
    eigvecs_sorted = eigenvector[:,idx]

    total_info = np.sum(np.abs(eigvals_sorted))

    cumulative = np.cumsum(np.abs(eigvals_sorted))
    threshold = percent_info * total_info
    num_components = np.searchsorted(cumulative, threshold) + 1

    eigvals_selected = eigvals_sorted[:num_components]
    Vs = eigvecs_sorted[:,:num_components]
    Vl = eigvecs_sorted[:,num_components:]
    
    output = {}
    output['num_components'] = num_components
    output['Vs'] = Vs
    output['Vl'] = Vl
    output['eigvals_sorted'] = eigvals_sorted
    output['eigvecs_sorted'] = eigvecs_sorted
    return output



def loss_stiff_subspace(phi, opt_object, diff_object, config):
    num_components = config['num_components']
    Vs = config['Vs']
    Vl = config['Vl']
    phi0 = config['phi0']
    
    params_aux = np.dot(Vs, phi).reshape(-1) + np.dot(Vl, phi0[num_components:]).reshape(-1)
    params = tuple(np.abs(params_aux).reshape(-1))    
    
    loss_val, frac_solutions_arr, rates_arr, _, gammas_predicted_arr = opt_object.objective_function_diff(params)
    return loss_val


def loss_and_grads_stiff_subspace(phi, opt_object, diff_object, config):
    num_components = config['num_components']
    Vs = config['Vs']
    Vl = config['Vl']
    phi0 = config['phi0']
    
    params_aux = np.dot(Vs, phi).reshape(-1) + np.dot(Vl, phi0[num_components:]).reshape(-1)
    params = tuple(np.abs(params_aux).reshape(-1))    
    
    loss_val, frac_solutions_arr, rates_arr, _, gammas_predicted_arr = opt_object.objective_function_diff(params)
    grad_val = diff_object.objective_function_grad(params, frac_solutions_arr, rates_arr, gammas_predicted_arr)
    grad_val_stiff = Vs.reshape(-1) @ grad_val.numpy()
    print("loss: ", loss_val, "grads: ", grad_val_stiff, "phi: ", phi)
    
    return loss_val, grad_val_stiff

In [ ]:
percent_info = 0.90

phiVec = np.linspace(0.87, 0.9, 10)
lossVec = np.zeros_like(phiVec)

config_dict = create_subspaces(params_default, percent_info, optimizer, diff)
phi0 = np.linalg.solve(config_dict['eigvecs_sorted'], np.array(params_default))

config_dict['phi0'] = phi0

In [ ]:

# res1 = loss_and_grads_stiff_subspace(0.87, optimizer, diff, config_dict)
# print(res1)
# res2 = loss_and_grads_stiff_subspace(0.87-1e-4, optimizer, diff, config_dict)
# print(res2)
# res3 = loss_and_grads_stiff_subspace(0.87+1e-4, optimizer, diff, config_dict)
# print(res3)


In [ ]:
# (res3[0]-res2[0])/(2e-4)

In [ ]:
# percent_info = 0.90

# phiVec = np.linspace(0.87, 0.9, 10)
# lossVec = np.zeros_like(phiVec)

# config_dict = create_subspaces(params_default, percent_info, optimizer, diff)
# phi0 = np.linalg.solve(config_dict['eigvecs_sorted'], np.array(params_default))

# config_dict['phi0'] = phi0

# lower_Vs = [0.0]
# upper_Vs = [1.0]

# res = sp.optimize.minimize(
#         lambda params: loss_and_grads_stiff_subspace(params, optimizer, diff, config_dict),
#         x0=phi0[:config_dict['num_components']],
#         jac=True,
#         bounds=[(a_i, b_i) for a_i, b_i in zip(lower_Vs, upper_Vs)],
#         method='L-BFGS-B')


In [ ]:


phiVec = np.linspace(0.895, 0.904, 15)
lossVec = np.zeros_like(phiVec)
gradsVec = np.zeros_like(phiVec)

for idx, phi in enumerate(phiVec):
    result = loss_and_grads_stiff_subspace(phi, optimizer, diff, config_dict)


loss:  194982.83066482755 grads:  13045.963399875855 phi:  0.895
loss:  194981.15892666008 grads:  13049.27210796104 phi:  0.8956428571428572
loss:  194979.48676265476 grads:  13052.581631682406 phi:  0.8962857142857144
loss:  194977.81417279592 grads:  13055.891969828956 phi:  0.8969285714285714
loss:  194976.14115694907 grads:  13059.203125087004 phi:  0.8975714285714286
loss:  194974.46771501112 grads:  13062.51509511873 phi:  0.8982142857142857
loss:  194972.7938468761 grads:  13065.827881682571 phi:  0.8988571428571429
loss:  194971.1195524418 grads:  13069.141484517402 phi:  0.8995
loss:  194969.44483163132 grads:  13072.455903240538 phi:  0.9001428571428571
loss:  194967.76968425224 grads:  13075.77113948455 phi:  0.9007857142857143
loss:  194966.0941103197 grads:  13079.087191666767 phi:  0.9014285714285715
loss:  194964.41810963952 grads:  13082.404060896928 phi:  0.9020714285714286
loss:  194962.7416821371 grads:  13085.721747813499 phi:  0.9027142857142857
loss:  194961.0648

In [ ]:


# percent_info = 0.90

# phiVec = np.linspace(0.87, 0.9, 10)
# lossVec = np.zeros_like(phiVec)

# config_dict = create_subspaces(params_default, percent_info, optimizer, diff)
# phi0 = np.linalg.solve(config_dict['eigvecs_sorted'], np.array(params_default))

# config_dict['phi0'] = phi0


# result = sp.optimize.minimize(lambda phi: loss_stiff_subspace(phi, optimizer, diff, config_dict), phi0[0],
#                             method='Nelder-Mead', options={'fatol': 1e-5})

# print(result)

# for idx, phi in enumerate(phiVec):
#     lossVec[idx] = loss_stiff_subspace(phi, optimizer, diff, config_dict)    
#     print(idx)
        

In [ ]:
# print(lossVec)

In [ ]:

# import matplotlib.pyplot as plt

# plt.figure()
# plt.plot(phiVec, lossVec)
# plt.yscale('log')
# plt.ylim(0.0, 0.5)
# plt.yscale('log')

In [ ]:
# eigenvalues, eigenvector = np.linalg.eigh(F_matrix)

# print(eigenvalues.real)

# print(eigenvector[-1])